In [1725]:
import numpy as np 
import pandas as pd
import copy
import pickle
import joblib

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler as under_sam

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

# Main implementation

In [1726]:
# loading the train set
df_train = pd.read_csv('EvalResources/KDDTrain+.txt', sep=",", header=None, skipinitialspace = True)
df_train = df_train[df_train.columns[:-1]]  # tags column
titles = pd.read_csv('EvalResources/Field Names.csv', header=None, skipinitialspace = True)
label = pd.Series(['label'], index=[41])
titles = pd.concat([titles[0], label])
df_train.columns = titles.to_list()
df_train = df_train.drop(['num_outbound_cmds'],axis=1)
df_train_original = df_train

# df_train_original.to_csv('KB Process/NSL-KDD Original Datasets/KDDTrain+_with_labels.txt', index=False)

#df_train_original

In [1727]:
# load test set
df_test = pd.read_csv('EvalResources/KDDTest+.txt', sep=",", header=None, skipinitialspace = True)
df_test_ = df_test.sort_index(axis=1)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
df_test = df_test.drop(['num_outbound_cmds'],axis=1)
df_test_original = df_test

# df_test_original.to_csv('KB Process/NSL-KDD Original Datasets/KDDTest+.txt', index=False)

#df_test_original

### Execution Parameters

In [1728]:
EXPORT_MODELS = 1
EXPORT_DATASETS = 0
EXPORT_PCA = 0
EXPORT_ENCODERS = 0

pd.options.display.max_columns = None

In [1729]:
# list of single attacks 
dos_attacks = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop', 'worm', 'apache2', 'mailbomb', 'processtable', 'udpstorm']
probe_attacks = ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']
r2l_attacks = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster',
                'snmpguess', 'spy', 'warezclient', 'httptunnel', 'named', 'sendmail', 'snmpgetattack', 'xlock', 'xsnoop']
u2r_attacks = ['buffer_overflow', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm'] 

# list of attack classes split according to detection layer
dos_probe_list = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop', 'ipsweep', 'nmap', 'portsweep', 'satan']
dos_probe_test = ['apache2', 'mailbomb', 'processtable', 'udpstorm', 'mscan', 'saint']
u2r_r2l_list = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster',
                'snmpguess', 'spy', 'warezclient', 'buffer_overflow', 'loadmodule', 'rootkit', 'perl']
u2r_r2l_test = ['httptunnel', 'named', 'sendmail', 'snmpgetattack', 'xlock', 'xsnoop', 'ps', 'xterm', 'sqlattack']
normal_list = ['normal']
categorical_features = ['protocol_type', 'service', 'flag']

In [1730]:
# load the features obtained with ICFS for both layer 1 and layer 2
with open('KBProcess/AWS Downloads/MinimalFeatures/NSL_features_l1.txt', 'r') as f:
    common_features_l1 = f.read().split(',')

with open('KBProcess/AWS Downloads/MinimalFeatures/NSL_features_l2.txt', 'r') as f:
    common_features_l2 = f.read().split(',')

In [1731]:
df_train_and_validate = copy.deepcopy(df_train_original)
df_test = copy.deepcopy(df_test_original)

In [1732]:
# Save all the targets for the dataset

y_test_l1 = [1 if x in (dos_attacks+probe_attacks) else 0 for x in df_test['label']]
y_test_l2 = [1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_test['label']]

if EXPORT_DATASETS:
    np.save("EvalResources/AdditionalSets/l1_full_test_targets.npy", y_test_l1)
    np.save("EvalResources/AdditionalSets/l2_full_test_targets.npy", y_test_l2)

# DOS PROBE CLASSIFIER

In [1733]:
# split in test and validation set for BOTH layers
df_train_original, df_val_original = train_test_split(df_train_and_validate, test_size=0.2, random_state=42)

# dataframes specifically for layer 1
df_train = copy.deepcopy(df_train_original)
df_val = copy.deepcopy(df_val_original)
df_test = copy.deepcopy(df_test_original)

# target variables for all layers
y_train_full = np.array([1 if x not in normal_list else 0 for x in df_train['label']])
y_test_full = np.array([1 if x not in normal_list else 0 for x in df_test['label']])

# set the target variables accordingly
y_train = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_train['label']])
y_validate = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_val['label']])
y_test = np.array([1 if x in (dos_attacks+probe_attacks) else 0 for x in df_test ['label']])

In [1734]:
# this dataframe contains the whole train set 
df_train = df_train.drop(['label'],axis=1)
df_train = df_train.reset_index().drop(['index'], axis=1)
#df_train

In [1735]:
# this dataframe contains the whole validation set
df_val = df_val.drop(['label'],axis=1)
df_val = df_val.reset_index().drop(['index'], axis=1)
#df_val

In [1736]:
# this dataframe contains the whole test set
df_test = df_test.drop(['label'],axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
#df_test

## Using features obtained with a random forest on numerical features only

# List of feature names
feature_names_l1 = ['src_bytes', 'logged_in', 'count', 'srv_count', 'srv_serror_rate',
       'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']

# Selecting features using loc
X_train = df_train.loc[:, feature_names_l1]
X_validate = df_val.loc[:, feature_names_l1]
X_test = df_test.loc[:, feature_names_l1]

In [1737]:
# Using the features extracted by using the ICFS algorithm
X_train = df_train[common_features_l1]
X_validate = df_val[common_features_l1]
X_test = df_test[common_features_l1]

In [1738]:
# 2 one-hot encoders, one for the features of layer1 and one for the features of layer2
ohe = OneHotEncoder(handle_unknown='ignore')
ohe2 = OneHotEncoder(handle_unknown='ignore')

In [1739]:
scaler1 = MinMaxScaler()

In [1740]:
scaler2 = MinMaxScaler()

In [1741]:
# scaling the train set for layer1
df_minmax = scaler1.fit_transform(X_train)
X_train = pd.DataFrame(df_minmax, columns=X_train.columns)

#X_train

In [1742]:
# scaling the validation set for layer1
df_minmax_val = scaler1.transform(X_validate)
X_validate = pd.DataFrame(df_minmax_val, columns=X_validate.columns)

#X_validate

In [1743]:
# scaling the test set for layer1
df_minmax_test = scaler1.transform(X_test)
X_test = pd.DataFrame(df_minmax_test, columns=X_test.columns)

#X_vtest

In [1744]:
# perform One-hot encoding for the train set
label_enc = ohe.fit_transform(df_train[categorical_features])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_train = pd.concat([X_train, df_enc], axis=1)

In [1745]:
# perform One-hot encoding for the validation set
label_enc = ohe.transform(df_val[categorical_features])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_validate = pd.concat([X_validate, df_enc], axis=1)

#X_validate

In [1746]:
# perform One-hot encoding for the test set
label_enc = ohe.transform(df_test[categorical_features])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)

#X_test

In [1747]:
print('Shape of the whole train set: ', X_train.shape)
print('Shape of its targets: ', y_train.shape)
print('Shape of the whole train set: ', X_validate.shape)
print('Shape of its targets: ', y_validate.shape)
print('Shape of the whole test set: ', X_test.shape)
print('Shape of its targets: ', y_test.shape)

Shape of the whole train set:  (100778, 101)
Shape of its targets:  (100778,)
Shape of the whole train set:  (25195, 101)
Shape of its targets:  (25195,)
Shape of the whole test set:  (22544, 101)
Shape of its targets:  (22544,)


# Export the dataset for training layer 1
if EXPORT_DATASETS:
    """
    X_train.to_csv('EvalResources/ProcessedDatasets/x_train_l1.txt', index=False)
    X_validate.to_csv('EvalResources/ProcessedDatasets/x_val_l1.txt', index=False)
    """
    np.save('EvalResources/ProcessedWithPCA/y_train_l1', y_train)
    np.save('EvalResources/ProcessedWithPCA/y_val_l1', y_validate)
    np.save('EvalResources/ProcessedWithPCA/y_test_l1', y_test)

### Principal Component Analysis

In [1748]:
pca_dos_probe = PCA(n_components=0.95)
X_train_dos_probe = pca_dos_probe.fit_transform(X_train)
X_test_dos_probe = pca_dos_probe.transform(X_test)
X_validate_dos_probe = pca_dos_probe.transform(X_validate)

# X_train = X_train.sort_index(axis=1)
X_train_dos_probe.shape

(100778, 28)

if EXPORT_PCA:
    # save the pca transformed as well as the transformer
    column_names = [f'PC{i}' for i in range(1, X_test_dos_probe.shape[1] + 1)]
    x = pd.DataFrame(data=X_test_dos_probe, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_test_l1_pca.txt', index=False)
    
    column_names = [f'PC{i}' for i in range(1, X_train_dos_probe.shape[1] + 1)]
    x = pd.DataFrame(data=X_train_dos_probe, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_train_l1_pca.txt', index=False)
    
    column_names = [f'PC{i}' for i in range(1, X_validate_dos_probe.shape[1] + 1)]
    x = pd.DataFrame(data=X_validate_dos_probe, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_val_l1_pca.txt', index=False)

### Building the classifier for the layer1

In [1749]:
from datetime import datetime

In [1750]:
# Voting classifiers

voting_classifiers = []

In [1751]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Using HistGradientBoosting classifier
dos_probe_classifier = HistGradientBoostingClassifier()

start = datetime.now()
dos_probe_classifier.fit(X_train_dos_probe, y_train)
ttime = datetime.now() - start

predicted = dos_probe_classifier.predict(X_test_dos_probe)

print('Metrics for layer 1:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Train time = ', ttime)

voting_classifiers.append(("hgbc", dos_probe_classifier))

Metrics for layer 1:
Confusion matrix: [TP FN / FP TN]
 [[12386   277]
 [ 2315  7566]]
Accuracy =  0.8850248403122782
F1 Score =  0.8537576167907921
Precision =  0.9646818819329338
Recall =  0.7657119724724218
Train time =  0:00:01.957621


# Using Random Forest Classifier
dos_probe_classifier = RandomForestClassifier()

start = datetime.now()
dos_probe_classifier.fit(X_train_dos_probe, y_train)
ttime = datetime.now() - start

predicted = dos_probe_classifier.predict(X_test_dos_probe)

print('Metrics for layer 1:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Train time = ', ttime)

voting_classifiers.append(("rf", dos_probe_classifier))

# Using the Naive Bayes Classifier
dos_probe_classifier = GaussianNB()

start = datetime.now()
dos_probe_classifier.fit(X_train_dos_probe, y_train)
ttime = datetime.now() - start

predicted = dos_probe_classifier.predict(X_test_dos_probe)

print('Metrics for layer 1:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Train time = ', ttime)
print('Shape of the train set for l1: ', X_train_dos_probe.shape)

voting_classifiers.append(("nbc", dos_probe_classifier))

# Voting classifier

from sklearn.ensemble import VotingClassifier

dos_probe_classifier = VotingClassifier(estimators=voting_classifiers, voting='soft')

start = datetime.now()
dos_probe_classifier.fit(X_train_dos_probe, y_train)
ttime = datetime.now() - start

predicted = dos_probe_classifier.predict(X_test_dos_probe)

print('Using a voting classifier:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Train time = ', ttime)
print('Shape of the train set for l1: ', X_train_dos_probe.shape)

# R2L+U2R classifier

In [1752]:
df_train = copy.deepcopy(df_train_original)
df_test = copy.deepcopy(df_test_original)
df_val = copy.deepcopy(df_val_original)

# load targeted attacks (Normal + r2l + u2r)
df_train = df_train[df_train['label'].isin(normal_list+u2r_attacks+r2l_attacks)]
df_val = df_val[df_val['label'].isin(normal_list+u2r_attacks+r2l_attacks)]
df_test = df_test[df_test['label'].isin(normal_list+u2r_attacks+r2l_attacks)]

# set the target variables accordingly
y_train = np.array([1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_train['label']])
y_val = np.array([1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_val['label']])
y_test = np.array([1 if x in (u2r_attacks+r2l_attacks) else 0 for x in df_test['label']])

df_train = df_train.drop(['label'],axis=1)
df_train = df_train.reset_index().drop(['index'], axis=1)
#df_train

In [1753]:
df_val = df_val.drop(['label'],axis=1)
df_val = df_val.reset_index().drop(['index'], axis=1)
#df_val

In [1754]:
df_test = df_test.drop(['label'],axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
#df_test

In [1755]:
X_train = df_train
X_validate = df_val
X_test = df_test

## Using features obtained with a random forest on numerical features only

# List of feature names
feature_names_l2 = ['count', 'is_guest_login', 'srv_count', 'hot', 'dst_host_serror_rate',
       'dst_host_srv_count', 'dst_host_count', 'dst_host_same_srv_rate',
       'dst_host_same_src_port_rate', 'num_file_creations', 'diff_srv_rate']

# Selecting features using loc
X_train = df_train.loc[:, feature_names_l2]
X_validate = df_val.loc[:, feature_names_l2]
X_test = df_test.loc[:, feature_names_l2]

In [1756]:
X_train = df_train[common_features_l2]
X_validate = df_val[common_features_l2]
X_test = df_test[common_features_l2]

In [1757]:
df_minmax = scaler2.fit_transform(X_train)
X_train = pd.DataFrame(df_minmax, columns=X_train.columns)
#X_train

In [1758]:
df_minmax = scaler2.transform(X_validate)
X_validate = pd.DataFrame(df_minmax, columns=X_validate.columns)
#X_validate

In [1759]:
df_minmax = scaler2.transform(X_test)
X_test = pd.DataFrame(df_minmax, columns=X_test.columns)
#X_test

In [1760]:
# perform One-hot encoding for the train set
label_enc = ohe2.fit_transform(df_train[categorical_features])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_train = pd.concat([X_train, df_enc], axis=1)
#X_train

In [1761]:
# perform One-hot encoding for the validation set
label_enc = ohe2.transform(df_val[categorical_features])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_validate = pd.concat([X_validate, df_enc], axis=1)
#X_validate

In [1762]:
# perform One-hot encoding for the test set
label_enc = ohe2.transform(df_test[categorical_features])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test = pd.concat([X_test, df_enc], axis=1)
#X_test

In [1763]:
print('Shape of the train set: ', X_train.shape)
print('Shape of its target: ', y_train.shape)
print('Shape of the test set: ', X_test.shape)
print('Shape of its target: ', y_test.shape)

Shape of the train set:  (54733, 51)
Shape of its target:  (54733,)
Shape of the test set:  (12663, 51)
Shape of its target:  (12663,)


In [1764]:
# Under sampling the train set for l2
sm = under_sam(sampling_strategy=1)
X_train, y_train = sm.fit_resample(X_train,y_train)

## Export the datasets
Train set has been scaled, one hot encoded, undersampled
Test set has been scaled and one hot encoded

# Export the dataset for training layer 2
if EXPORT_DATASETS:
    # X_train.to_csv('EvalResources/ProcessedDatasets/x_train_l2.txt', index=False)
    np.save('EvalResources/ProcessedWithPCA/y_train_l2', y_train)
    # X_validate.to_csv('EvalResources/ProcessedDatasets/x_val_l2.txt', index=False)
    np.save('EvalResources/ProcessedWithPCA/y_val_l2', y_val)
    np.save('EvalResources/ProcessedWithPCA/y_test_l2', y_test)

In [1765]:
# Principal Component Analysis
pca_r2l_u2r = PCA(n_components=0.95)
X_train_r2l_u2r = pca_r2l_u2r.fit_transform(X_train)
X_test_r2l_u2r = pca_r2l_u2r.transform(X_test)
X_validate_r2l_u2r = pca_r2l_u2r.transform(X_validate)

# Using Random Forest Classifier
r2l_u2r_classifier = RandomForestClassifier()

start = datetime.now()
r2l_u2r_classifier.fit(X_train_r2l_u2r, y_train)
ttime = datetime.now() - start

predicted = r2l_u2r_classifier.predict(X_test_r2l_u2r)

print('Metrics for layer 2:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Matthew corr = ', matthews_corrcoef(y_test,predicted))
print('Train time = ', ttime)
print('Shape of the training set: ', X_train_r2l_u2r.shape)

In [1766]:
# Support Vector Machine for layer l2
r2l_u2r_classifier = SVC(C=0.1, gamma=0.01, kernel='rbf', probability=True)

#r2l_u2r_classifier = SVC()

start = datetime.now()
r2l_u2r_classifier.fit(X_train_r2l_u2r, y_train)
ttime = datetime.now() - start

predicted = r2l_u2r_classifier.predict(X_test_r2l_u2r)

print('Metrics for layer 2:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_test,predicted))
print('Accuracy = ', accuracy_score(y_test,predicted))
print('F1 Score = ', f1_score(y_test,predicted))
print('Precision = ', precision_score(y_test,predicted))
print('Recall = ', recall_score(y_test,predicted))
print('Matthew corr = ', matthews_corrcoef(y_test,predicted))
print('Train time = ', ttime)
print('Shape of the training set: ', X_train_r2l_u2r.shape)

Metrics for layer 2:
Confusion matrix: [TP FN / FP TN]
 [[9092  619]
 [1550 1402]]
Accuracy =  0.8287135749822317
F1 Score =  0.5638447617132515
Precision =  0.6937159821870361
Recall =  0.4749322493224932
Matthew corr =  0.47472053754163585
Train time =  0:00:00.523809
Shape of the training set:  (1624, 13)


In [1767]:
predicted = r2l_u2r_classifier.predict(X_train_r2l_u2r)

print('Metrics for layer 2:')
print('Confusion matrix: [TP FN / FP TN]\n', confusion_matrix(y_train,predicted))
print('Accuracy = ', accuracy_score(y_train,predicted))
print('F1 Score = ', f1_score(y_train,predicted))
print('Precision = ', precision_score(y_train,predicted))
print('Recall = ', recall_score(y_train,predicted))
print('Matthew corr = ', matthews_corrcoef(y_train,predicted))

Metrics for layer 2:
Confusion matrix: [TP FN / FP TN]
 [[694 118]
 [  5 807]]
Accuracy =  0.9242610837438424
F1 Score =  0.9291882556131261
Precision =  0.8724324324324324
Recall =  0.9938423645320197
Matthew corr =  0.8568597990061858


if EXPORT_PCA:
        # save the pca transformed as well as the transformer
    column_names = [f'PC{i}' for i in range(1, X_test_r2l_u2r.shape[1] + 1)]
    x = pd.DataFrame(data=X_test_r2l_u2r, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_test_l2_pca.txt', index=False)
        
    print(x.shape)
    
    column_names = [f'PC{i}' for i in range(1, X_train_r2l_u2r.shape[1] + 1)]
    x = pd.DataFrame(data=X_train_r2l_u2r, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_train_l2_pca.txt', index=False)
        
    print(x.shape)
    
    column_names = [f'PC{i}' for i in range(1, X_validate_r2l_u2r.shape[1] + 1)]
    x = pd.DataFrame(data=X_validate_r2l_u2r, columns=column_names)
    x.to_csv('EvalResources/ProcessedWithPCA/x_val_l2_pca.txt', index=False)
        
    print(x.shape)

### Export the classifiers

In [1768]:
if EXPORT_MODELS:
    with open('EvalResources/Models/HGBC/l1_classifier.pkl', "wb") as f:
        pickle.dump(dos_probe_classifier, f)
    with open('EvalResources/Models/HGBC/l2_classifier.pkl', "wb") as f:
        pickle.dump(r2l_u2r_classifier, f)

# Export one hot encoders
if EXPORT_ENCODERS:
    joblib.dump(ohe, 'EvalResources/Encoders/ohe_l1.pkl')
    joblib.dump(ohe2, 'EvalResources/Encoders/ohe_l2.pkl')
    joblib.dump(scaler1, 'EvalResources/Encoders/Scaler_l1.pkl')
    joblib.dump(scaler2, 'EvalResources/Encoders/Scaler_l2.pkl')
    joblib.dump(pca_dos_probe, 'EvalResources/Encoders/pca_transformer_l1.pkl')
    joblib.dump(pca_r2l_u2r, 'EvalResources/Encoders/pca_transformer_l2.pkl')

# Testing

In [1625]:
df_test1 = copy.deepcopy(df_test_original)
df_test2 = copy.deepcopy(df_test_original)

y_test_real = np.array([0 if x=='normal' else 1 for x in df_test1['label']])

np.save("EvalResources/Test/KDDTest+_targets", y_test_real)

In [1626]:
y_test_real.shape

(22544,)

X_test1 = df_test1.loc[:, feature_names_l1]

In [1627]:
X_test1 = df_test1[common_features_l1]

In [1628]:
df_minmax = scaler1.transform(X_test1)
X_test1 = pd.DataFrame(df_minmax, columns=X_test1.columns)
label_enc = ohe.transform(df_test1.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test1 = pd.concat([X_test1, df_enc], axis=1)

X_test_layer1 = pca_dos_probe.transform(X_test1)
print('Test set shape for layer 1: ', X_test_layer1.shape)

Test set shape for layer 1:  (22544, 28)


In [1629]:
#X_test1

In [1630]:
X_test2 = df_test2

X_test2 = df_test2.loc[:, feature_names_l2]

In [1631]:
X_test2 = df_test2[common_features_l2] 

In [1632]:
df_minmax = scaler2.transform(X_test2)
X_test2 = pd.DataFrame(df_minmax, columns=X_test2.columns)
label_enc = ohe2.transform(df_test2.iloc[:,1:4])
label_enc.toarray()
new_labels = ohe2.get_feature_names_out(categorical_features)
df_enc = pd.DataFrame(data=label_enc.toarray(), columns=new_labels)
X_test2 = pd.concat([X_test2, df_enc], axis=1)

X_test_layer2 = pca_r2l_u2r.transform(X_test2)
print('Test set shape for layer 1: ', X_test_layer1.shape)
print('Test set shape for layer 2: ', X_test_layer2.shape)

Test set shape for layer 1:  (22544, 28)
Test set shape for layer 2:  (22544, 13)


In [1633]:
#X_test_layer1

In [1634]:
#X_test_layer2

In [1635]:
# same classifiers obtained above
classifier1 = dos_probe_classifier
classifier2 = r2l_u2r_classifier

# test the other classifiers obtained in the evaluation phase
#classifier1 = joblib.load("EvalResources/Models/l1_clf.pkl")
#classifier2 = joblib.load("EvalResources/Models/l2_clf.pkl")

In [1636]:
import warnings

# Suppressing the warning
warnings.filterwarnings("ignore", message="X does not have valid feature names.*")

result = []

start = datetime.now()
for i in range(X_test_layer2.shape[0]):
    layer1 = classifier1.predict(X_test_layer1[i].reshape(1, -1))[0]
    if layer1 == 1:
        result.append(layer1)
    else:
        layer2 = classifier2.predict(X_test_layer2[i].reshape(1, -1))[0]
        if layer2 == 1:
            result.append(layer2)
        else:
            result.append(0)
clf_time = datetime.now() - start

result = np.array(result)

KeyboardInterrupt: 

In [ ]:
# the results may vary
# C=0.1, gamma=0.01
print('Results for the full pipeline:')
print(confusion_matrix(y_test_real,result))
print('Accuracy = ', accuracy_score(y_test_real,result))
print('F1 Score = ', f1_score(y_test_real,result))
print('Precision = ', precision_score(y_test_real,result))
print('Recall = ', recall_score(y_test_real,result))
print('Matthew corr = ', matthews_corrcoef(y_test_real,result))
print('Classification time = ', clf_time)

In [ ]:
result = []
TP = FP = TN = FN = 0

start = datetime.now()
for i in range(X_test_layer2.shape[0]):
    layer1 = classifier1.predict(X_test_layer1[i].reshape(1, -1))[0]
    if layer1 == 1:
        result.append(layer1)
        if y_test_real[i] == 1:
            TP += 1
        else:
            FP += 1
    else:
        layer2 = classifier2.predict(X_test_layer2[i].reshape(1, -1))[0]
        if layer2 == 1:
            result.append(layer2)
            if y_test_real[i] == 1:
                TP += 1
            else:
                FP += 1
        else:
            result.append(0)
            if y_test_real[i] == 0:
                TN += 1
            else:
                FN += 1
clf_time = datetime.now() - start

result = np.array(result)

# Compute TPR, FPR, TNR, FNR
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
FNR = FN / (FN + TP)

# Print TPR, FPR, TNR, FNR
print('True Positive Rate (TPR) =', TPR)
print('False Positive Rate (FPR) =', FPR)
print('True Negative Rate (TNR) =', TNR)
print('False Negative Rate (FNR) =', FNR)
print('Classification time = ', clf_time)

In [ ]:
# ROC graph

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test_real, result)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

plt.savefig('static_roc_curve-hgbc.png')

plt.show()

### Export the test sets

if True:
    column_names = [f'PC{i}' for i in range(1, X_test_layer1.shape[1] + 1)]
    X1_test = pd.DataFrame(data=X_test_layer1, columns=column_names)
    X1_test.to_csv('EvalResources/AdditionalSets/x_test_l1_pca.txt', index=False)
    
    column_names = [f'PC{i}' for i in range(1, X_test_layer2.shape[1] + 1)]
    X2_test = pd.DataFrame(data=X_test_layer2, columns=column_names)
    X2_test.to_csv('EvalResources/AdditionalSets/x_test_l2_pca.txt', index=False)
    
    np.save('EvalResources/AdditionalSets/y_test', y_test_real)

### evaluate seen and unseen attack categories

In [ ]:
# load testset
df_test = pd.read_csv('EvalResources/KDDTest+.txt', sep=",", header=None, skipinitialspace = True)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
y_test = df_test['label']
df_test = df_test.drop(['num_outbound_cmds'],axis=1)

df_test_original = df_test

In [ ]:
if EXPORT_DATASETS:
    df_test_original.to_csv('EvalResources/ProcessedDatasets/x_test_full.txt', index=False)
    np.save('EvalResources/ProcessedDatasets/y_test_full', y_test)
    
#df_test_original

In [ ]:
new_attack = []
for i in df_test_original['label'].value_counts().index.tolist()[1:]:
    if i not in df_train_original['label'].value_counts().index.tolist()[1:]:
        new_attack.append(i)
        
new_attack.sort()
#new_attack

In [ ]:
index_of_new_attacks = []

for i in range(len(df_test_original)):
    if df_test_original['label'][i] in new_attack:
        index_of_new_attacks.append(df_test_original.index[i])

In [ ]:
#len(index_of_new_attacks)

In [ ]:
new_attack.append('normal')
#new_attack

In [ ]:
index_of_old_attacks = []

for i in range(len(df_test_original)):
    if df_test_original['label'][i] not in new_attack:
        index_of_old_attacks.append(df_test_original.index[i])

In [ ]:
print('Number of new attacks in the test set: ', result[index_of_new_attacks].shape[0])
print('Number of new attacks detected by the classifiers: ', result[index_of_new_attacks].sum())
print('Proportion of new attacks detected: ', result[index_of_new_attacks].sum()/result[index_of_new_attacks].shape[0])

In [ ]:
print('Number of old attacks in the test set: ', result[index_of_old_attacks].shape[0])
print('Number of old attacks detected by the classifiers: ', result[index_of_old_attacks].sum())
print('Proportion of old attacks detected: ', result[index_of_old_attacks].sum()/result[index_of_old_attacks].shape[0])

### Evaluate single attack types

In [ ]:
# load test set
df_test = pd.read_csv('EvalResources/KDDTest+.txt', sep=",", header=None, skipinitialspace = True)
df_test = df_test[df_test.columns[:-1]]
df_test.columns = titles.to_list()
y_test = df_test['label']
df_test = df_test.drop(['num_outbound_cmds'],axis=1)
df_test_original = df_test
df = df_test_original

dos_index = df.index[(df['label'].isin(dos_attacks))].tolist()
probe_index = df.index[(df['label'].isin(probe_attacks))].tolist()
r2l_index = df.index[(df['label'].isin(r2l_attacks))].tolist()
u2r_index = df.index[(df['label'].isin(u2r_attacks))].tolist()

print('Evaluation split into single attack type:')
print("Number of dos attacks: ", result[dos_index].shape[0])
print("Number of detected attacks: ", result[dos_index].sum())
print("Ratio of detection: ", result[dos_index].sum()/result[dos_index].shape[0])

print("Number of probe attacks: ", result[probe_index].shape[0])
print("Number of detected attacks: ", result[probe_index].sum())
print("Ratio of detection: ", result[probe_index].sum()/result[probe_index].shape[0])

print("Number of r2l attacks: ", result[r2l_index].shape[0])
print("Number of detected attacks: ", result[r2l_index].sum())
print("Ratio of detection: ", result[r2l_index].sum()/result[r2l_index].shape[0])

print("Number of u2r attacks: ", result[u2r_index].shape[0])
print("Number of detected attacks: ", result[u2r_index].sum())
print("Ratio of detection: ", result[u2r_index].sum()/result[u2r_index].shape[0])